# ============================================
# 🧠 GEMMA 3-12B — FINE-TUNING COM IMAGENS
# ============================================

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/UFF/6_periodo/'Modelos de Linguagem Neurais'/DIMEMEX/


dataset  LICENSE  notebooks  README.md


In [ ]:
%cd /content/drive/MyDrive/UFF/6_periodo/'Modelos de Linguagem Neurais'/DIMEMEX/

/content/drive/MyDrive/UFF/6_periodo/Modelos de Linguagem Neurais/DIMEMEX


In [ ]:
from datasets import concatenate_datasets
import json
import pandas as pd
from datasets import Dataset
from pathlib import Path

def load_split(split_name):
    img_dir = Path(split_name)
    with open(f"{split_name}_data.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    df["label"] = pd.read_csv(f"{split_name}_labels.csv").iloc[:, 0]
    df["image_path"] = df["MEME-ID"].apply(lambda x: str(img_dir / x))
    return Dataset.from_pandas(df)

train_ds = load_split("train")
val_ds = load_split("validation")
test_ds = load_split("test")

dataset_dict = {"train": train_ds, "validation": val_ds, "test": test_ds}


In [ ]:
# ============================================
# 🛠️ 1. Instalar dependências
# ============================================
!pip install -q transformers accelerate datasets peft bitsandbytes torch torchvision pillow tqdm scikit-learn matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.7 MB/s eta 0:00:00


In [ ]:
# ============================================
# 🗂️ 2. Upload dos dados
# ============================================
from google.colab import files
import zipfile, os

print("📁 Envie o CSV (com colunas image_path,label)...")
uploaded = files.upload()
csv_name = list(uploaded.keys())[0]

print("\n📦 Agora envie o ZIP com as imagens (ex: DIMEMEX_images.zip)...")
uploaded2 = files.upload()
zip_name = list(uploaded2.keys())[0]

os.makedirs("DIMEMEX_images", exist_ok=True)
with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall("DIMEMEX_images")

print("✅ Arquivos carregados e extraídos.")

📁 Envie o CSV (com colunas image_path,label)...


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# ============================================
# ⚙️ 3. Configuração e análise do dataset
# ============================================

In [ ]:
import pandas as pd

DATA_CSV = csv_name
IMAGE_ROOT = "DIMEMEX_images"

df = pd.read_csv(DATA_CSV)
df["image_path"] = df["image_path"].apply(
    lambda p: os.path.join(IMAGE_ROOT, p) if not os.path.isabs(p) else p
)
df = df[df["image_path"].apply(os.path.exists)].reset_index(drop=True)

print("\n📊 Tamanho total do dataset:", len(df))
print(df["label"].value_counts().rename_axis("Label").reset_index(name="Quantidade"))


In [ ]:
# ============================================
# ⚙️ 4. Configurações principais
# ============================================

In [ ]:
import torch
from PIL import Image
from tqdm import tqdm
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from transformers import (
    AutoProcessor,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_ID = "HuggingFaceTB/SmolVLM-256M-Instruct"
NUM_LABELS = 2
BATCH_SIZE = 2
EPOCHS = 2
LR = 2e-4
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
# ============================================
# 📦 5. Preparar datasets de treino, val e teste
# ============================================
# train_df, test_df = train_test_split(df, test_size=0.15, stratify=df["label"], random_state=42)
# train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df["label"], random_state=42)

ds_train = Dataset.from_pandas(train_df.reset_index(drop=True))
ds_val = Dataset.from_pandas(val_df.reset_index(drop=True))
ds_test = Dataset.from_pandas(test_df.reset_index(drop=True))

print(f"Treino: {len(ds_train)} | Validação: {len(ds_val)} | Teste: {len(ds_test)}")

In [ ]:
# ============================================
# 🧩 6. Processor e pré-processamento de imagem
# ============================================
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(MODEL_ID)

def preprocess(examples):
    images = [Image.open(p).convert("RGB") for p in examples["image_path"]]
    inputs = processor(images=images, return_tensors="pt")
    inputs["labels"] = examples["label"]
    return inputs

ds_train = ds_train.map(preprocess, batched=True, remove_columns=ds_train.column_names)
ds_val = ds_val.map(preprocess, batched=True, remove_columns=ds_val.column_names)
ds_test = ds_test.map(preprocess, batched=True, remove_columns=ds_test.column_names)

In [ ]:
# ============================================
# 🧠 7. Modelo SmolVLM-256M-Instruct + LoRA
# ============================================
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    load_in_8bit=True,
)

base_model = prepare_model_for_kbit_training(base_model)

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()

In [ ]:
# ============================================
# 📈 8. Métricas de avaliação
# ============================================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted", zero_division=0),
        "recall": recall_score(labels, preds, average="weighted", zero_division=0),
    }

In [ ]:
# ============================================
# 🧮 9. Configurações de treino
# ============================================
training_args = TrainingArguments(
    output_dir="./SmolVLM_256M_Instruct_only_lora",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    load_best_model_at_end=True,
    logging_steps=50,
    save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    tokenizer=processor,
    compute_metrics=compute_metrics,
)

NameError: name 'TrainingArguments' is not defined

In [ ]:
# ============================================
# 🚀 10. Treinar modelo
# ============================================
trainer.train()

In [ ]:
# ============================================
# 📊 11. Diagnóstico — Loss, Accuracy e F1
# ============================================
import matplotlib.pyplot as plt
import json

log_file = os.path.join(training_args.output_dir, "trainer_state.json")

if os.path.exists(log_file):
    with open(log_file, "r") as f:
        logs = json.load(f)

    train_logs = logs.get("log_history", [])

    train_steps = [x["step"] for x in train_logs if "loss" in x]
    train_loss = [x["loss"] for x in train_logs if "loss" in x]

    eval_steps = [x["step"] for x in train_logs if "eval_loss" in x]
    eval_loss = [x["eval_loss"] for x in train_logs if "eval_loss" in x]
    eval_acc = [x["eval_accuracy"] for x in train_logs if "eval_accuracy" in x]
    eval_f1 = [x["eval_f1"] for x in train_logs if "eval_f1" in x]

    plt.figure(figsize=(8,4))
    plt.plot(train_steps, train_loss, label="Train Loss")
    plt.plot(eval_steps, eval_loss, label="Eval Loss")
    plt.title("Evolução da Loss por Época")
    plt.xlabel("Etapas (steps)")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    plt.figure(figsize=(8,4))
    plt.plot(eval_steps, eval_acc, label="Accuracy", marker='o')
    plt.plot(eval_steps, eval_f1, label="F1 Score", marker='s')
    plt.title("Desempenho por Época")
    plt.xlabel("Etapas (steps)")
    plt.ylabel("Pontuação")
    plt.legend()
    plt.show()
else:
    print("⚠️ Nenhum log encontrado. O treino pode não ter sido salvo corretamente.")

In [ ]:
# ============================================
# ✅ 12. Avaliação final e resumo de métricas
# ============================================
results = trainer.evaluate(ds_test)
print("\n📊 Resultados finais no conjunto de teste:")
for k, v in results.items():
    print(f"{k:20s}: {v:.4f}")

In [ ]:
# ============================================
# 💾 13. Salvar modelo final
# ============================================
model.save_pretrained("SmolVLM_256M_Instruct_only_lora")
processor.save_pretrained("SmolVLM_256M_Instruct_only_lora")

print("\n✅ Fine-tuning concluído e pesos salvos em: SmolVLM_256M_Instruct_only_lora/")
